In [1]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from scipy.optimize import minimize_scalar, BFGS

## Simplifying mixed-integer optimization problems with the derivate trick

This notebook illustrates a simple technique which can be used to reduce the number of continuous parameters in mixed integer optimization problems. We use this technique to convert an instance of the unit commitment problem with $n$ units to a problem with a single continuous parameter and $n$ binary parameters. This problem can then be solved with a Quantum-Classical optimization techinque, in which the continuous parameter is optimized over in a classical outer loop, while the binary parameters are fixed using a quantum computer. The problem of solving for optimal binary parameters given fixed continuous parameters is then equivalent the knapsack problem, which is discussed in a separate notebook. 

### The unit commitment problem

We consider the single-timestep unit commitment problem with $n$ units. This problem is specified by:
* A target load L. 
* A list of $n$ units, each of which has:
    - Max and min power levels $p_{i, min}$ and $p_{i, max}$.
    - A parameter $A_i$ giving the cost of turning the unit on.
    - Parameters $B_i$ and $C_i$ which define a quadratic cost function giving the cost of increasing the units power output once it is on. 

Associated with each unit is a binary variable $y_i$ specifying whether or not the unit was turned on, and a continuous variable $p_i$ giving the power output by the unit if it is turned on. The problem is to find a setting of these variables which produces a total power of at least $L$, while minimizing the overall cost. Formally we want to solve: 

$$
\begin{align} 
\min( \sum_i A_i y_i + B_i p_i + C_i p_i^2) \ \ s.t. \ \   &\sum_i p_i \geq L  \ \ \text{ and } \\
&y_i \cdot p_{i, min} \leq p_i \leq y_i \cdot p_{i, max}  \ \forall \ i.
\end{align}
 $$

Next, we introduce some simple code to generate random instances of the unit commitment problem. 


In [2]:
def generate_units(N, A_range=(10, 50), B_range=(0.5, 1.5), C_range=(0.01, 0.2), p_min_range=(10, 20), p_max_range=(50, 100)):
    """
    Generate N power units with random characteristics.
    """
    A = np.random.uniform(*A_range, size=N)  # Linear fixed cost coefficients
    B = np.random.uniform(*B_range, size=N)  # Linear operational cost coefficients
    C = np.random.uniform(*C_range, size=N)  # Quadratic cost coefficients
    p_min = np.random.uniform(*p_min_range, size=N)  # Minimum power outputs
    p_max = np.random.uniform(*p_max_range, size=N)  # Maximum power outputs

    return np.array(A), np.array(B), np.array(C), np.array(p_min), np.array(p_max)

class UnitCommitmentProblem:
    def __init__(self, n_units=10, load_factor=0.5):
        """
        Initialize the Unit Commitment Problem.
        
        :param n_units: Number of units to generate
        :param load_factor: Fraction of maximum power to use as load
        """
        # Generate units with their characteristics
        self.A, self.B, self.C, self.p_min, self.p_max = generate_units(N=n_units)
        
        # Calculate total load
        self.L = np.sum(self.p_max) * load_factor
    
    def set_load_factor(self, load_factor):
        """
        Set the load factor and recalculate total load.
        
        :param load_factor: Fraction of maximum power to use as load
        """
        self.L = np.sum(self.p_max) * load_factor

    def evaluate_solution(self, y_i, p_i):
        """
        Evaluate a given solution.
        
        :param y_i: Binary array indicating unit commitment (1 if on, 0 if off)
        :param p_i: Array of power outputs for each unit
        :return: Total cost of the solution, Bool indicating if solution is valid.
        """
        fixed_cost = np.sum(self.A * y_i)
        operational_cost = np.sum(self.B * p_i + self.C * p_i**2)
        total_cost = fixed_cost + operational_cost
        
        # Check constraints
        invalid = False
        total_power = np.sum(p_i)
        if total_power < self.L:
            invalid = True

        for i in range(len(y_i)):
            if p_i[i] < y_i[i] * self.p_min[i] or p_i[i] > y_i[i] * self.p_max[i]:
                invalid = True

        return total_cost, invalid

In practice, commercial solvers can quickly solve instances of the single timestep unit commitment problem with up to 100 units. 

In [3]:
def gurobi_UC_solver(uc, verbose=False):
    """
    Solve the Unit Commitment problem using a quadratic programming solver (Gurobi).
    
    Args:
        UCProblem (Unit Commitment Problem): An instance of the UnitCommitmentProblem class.

    Returns:
        bitstirng (string): Binary solution (on/off for each unit).
        p_solution (list): Power output solution for each unit.
        cost (float): Total cost of operation.
        runtime (float): Time taken to solve the problem.
    """
    
    n_units = len(uc.A)  # Number of power units

    # Create a new model
    model = gp.Model("unit_commitment")

    # Suppress output if verbose is False
    if not verbose:
        model.setParam('OutputFlag', 0)

    # Add binary decision variables for turning the units on/off
    y = model.addVars(n_units, vtype=GRB.BINARY, name="y")

    # Add continuous decision variables for the power output of each unit
    p = model.addVars(n_units, lb=0, name="p")

    # Objective function: minimize total cost
    total_cost = gp.quicksum( (uc.A[i]*y[i]) + (uc.B[i]*p[i]) + (uc.C[i]*p[i]*p[i]) for i in range(n_units))
    model.setObjective(total_cost, GRB.MINIMIZE)

    # Add constraint: Total power output must meet the load demand L
    model.addConstr(gp.quicksum(p[i] for i in range(n_units)) == uc.L, name="power_balance")

    # Add constraints: Power output of each unit must respect on/off state and power bounds
    for i in range(n_units):
        model.addConstr(p[i] >= uc.p_min[i] * y[i], name=f"min_power_{i}")
        model.addConstr(p[i] <= uc.p_max[i] * y[i], name=f"max_power_{i}")

    # Increase precision in Gurobi
    model.setParam('MIPGap', 1e-12)  # Tighter gap for optimality
    model.setParam('FeasibilityTol', 1e-6)  # Tighter feasibility tolerance

    # Optimize the model
    model.optimize()

    if model.status == GRB.OPTIMAL:
        # Extract binary (y) and continuous (p) solutions
        y_solution = model.getAttr("X", y)  
        p_solution = model.getAttr("X", p)
        total_cost = model.objVal

        y_solution = ''.join('1' if val > 0.5 else '0' for i,val in y_solution.items()) # convert to string
        p_solution = [p_solution[i] for i in range(len(p_solution))] # convert to list

        results = {}
        results['bitstring'] = y_solution
        results['power'] = p_solution
        results['cost'] = total_cost
        results['runtime'] = model.Runtime

        return results
    else:
        raise ValueError("Optimization failed")



# Initialize a random instance, and solve it with Gurobi
uc = UnitCommitmentProblem(n_units=100, load_factor=0.2)
results = gurobi_UC_solver(uc, verbose=True)

# print("Gurobi Solution has total cost:", results['cost'])

Restricted license - for non-production use only - expires 2026-11-23
Set parameter MIPGap to value 1e-12
Set parameter FeasibilityTol to value 1e-06
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 8845HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
MIPGap  1e-12

Optimize a model with 201 rows, 200 columns and 500 nonzeros
Model fingerprint: 0xfc6d1404
Model has 100 quadratic objective terms
Variable types: 100 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [5e-01, 5e+01]
  QObjective range [2e-02, 4e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+03, 1e+03]
Found heuristic solution: objective 13423.306257
Presolve time: 0.01s
Presolved: 401 rows, 300 columns, 1000 nonzeros
Presolved model has 100 quadratic objective terms
Varia

Even those these instances are classically solvable, it is still interesting to consider alternate techniques for solving this problem, as they may generalize to more complicated, classically intractable problems. We move on to considering this task next. 

### Hybrid Solutions to the UC Problem

Near-term quantum computers are unlikely to be able to encode the continuous variables present in the unit commitment problem. Consequently, attempts at solving the unit commitment problem on a quantum computer will rely on hybrid techniques, in which a classical optimization technique fixes the continuous parameters in the problem, and another, possibly quantum, optimization techinque fixes the problem's binary variables. Ideally this hybrid approach results in a binary optimization problem which preserves some of of the unit commitment problem's original hardness, while reducing the ammount of data that would need to be encoded on the quantum computer. 

We first describe a simple hyrbid approach to solving the unit commitment problem, then refine it. We first make a small rewrite to formulation of the UC problem, where the continuous variables in the problem give the power output of each unit *if it is turned on*, while the binary variables describe which units are turned on. Letting $\xi$ denote the optimal solution to the UC problem, we then have 

$$
\begin{align*} 
\xi = \min_{p_i}\left( \min_{y_i} \left( \sum_i A_i y_i + B_i p_i y_i + C_i p_i^2 y_i \right) \right) \ \ s.t. \ \   &\sum_i p_i y_i \geq L  \ \ \text{ and } \\
& p_{i, min} \leq p_i \leq p_{i, max} \ \forall \ i.
\end{align*}
 $$

This formulation is very close to the original formulation of the UC problem, but now a power level $p_i$ can be set even if a unit is turned off, and the product $p_i y_i$ gives the actual power output of each unit. 

Critically this formulation also reveals a "two-loop" structure to the unit commitment problem. The outer loop involves a minimization of the continous variables $p_i$, while the inner loop minimizes over the binary $y_i$ variables. Once $p_i$ variables have been set, the remaining problem is a purely binary optimization problem -- which is equivalent to the knapsack problems after a few sign flips. 

But this has exactly the form we want: a classical and continous outer loop, along with a binary inner loop, which retains some theoretical hardness (although easily solved in practice)! The only problem is that the outer loop involves optimizing over $n$ continous variables. This grows with the number of units, meaning the outer loop quickly becomes infeasible. 

We refine this approach using some calculus. First order optimality conditions give that any optimal solution can be described by parameters $p_i$ and a continous parameter $D$ satisfying: 

$$
\begin{align*}
\forall i : \ &B_i + 2 C_i p_i = D \ \ \textbf{ or }  \\ &B_i + 2 C_i p_{i, max} < D \ \text{ and } \ p_i = p_{i, max} \ \  \textbf{ or }\\
&B_i + 2 C_i p_{i, min} < D \ \text{ and } \ p_i = p_{i,min}.
\end{align*}
$$

A proof of this is given in the full paper. Intuitively, it can be understood by noting that if two $p_i$'s not satisfying this condition are included in the solution, a better solution could be found by infintessimally raising the value of some $p_i$ and lowering the value of another. 

With this observation in hand, we can rewrite the optimal parameter $\xi$ as 

$$
\begin{align*} 
\xi = \min_{D}\left( \min_{y_i} \left( \sum_i A_i y_i + B_i p_i y_i + C_i p_i^2 y_i \right) \right) \ \ s.t. \ \   &\sum_i p_i y_i \geq L  \ \ \text{ and } \ \forall i \\
& p_i =  \min\left(\max\left(\frac{D - B_i}{2 C_i},p_{i,min}\right), p_{i,max}\right).
\end{align*}
 $$

 This optimization now has exactly the form we want. An outer optimization loop involving optimization over a continous outer parameter, along with an optimization of a binary inner parameter. Now we impliment this optimization in code. 

In [6]:
class ReducedUnitCommitmentProblem:
    def __init__(self, n_units, unit_powers, unit_costs, load):
        """
        Initialize the Unit Commitment Problem.
        
        :param n_units: Number of units to generate
        :param load_factor: Fraction of maximum power to use as load
        """
        # Generate units with their characteristics
        self.n_units, self.unit_powers, self.unit_costs, self.load = n_units, unit_powers, unit_costs, load

def reduceUC(uc, D):
    """
    Reduce the Unit Commitment problem to a knapsack problem by fixing the continuous parameter D describing first derivates with respect to unit's power. 
    
    Args:
        uc (UnitCommitmentProblem): An instance of the UnitCommitmentProblem class.
        D (float): The continous parameter describing first derivates with respect to unit's power.
    Returns:
        reduced_uc (ReducedUnitCommitmentProblem): An instance of the ReducedUnitCommitmentProblem class.
    """

    n_units = len(uc.A)
    unit_powers = []
    unit_costs = []

    for i in range(n_units):
        power = (D - uc.B[i]) / (2 * uc.C[i])
        if power < uc.p_min[i]:
            power = uc.p_min[i]
        elif power > uc.p_max[i]:
            power = uc.p_max[i]
        cost = uc.A[i] + uc.B[i] * power + uc.C[i] * power * power 
        unit_powers.append(power)
        unit_costs.append(cost)

    reduced_uc = ReducedUnitCommitmentProblem(n_units=n_units, unit_powers=unit_powers, unit_costs=unit_costs, load=uc.L)

    return reduced_uc


# For now we define a classical (Gurobi) solver for the reduced UC problem. Eventually this can be replaces with a quantum solver. 
def gurobi_reduced_UC_solver(reduced_uc, verbose=False):
    """
    Solve the reduced Unit Commitment problem using a quadratic programming solver (Gurobi).
    Args:
        reduced_uc (ReducedUnitCommitmentProblem): An instance of the ReducedUnitCommitmentProblem class.
    Returns:
        bitstirng (string): Binary solution (on/off for each unit).
        cost (float): Total cost of operation.
        runtime (float): Time taken to solve the problem.
    """

    n_units = reduced_uc.n_units  # Number of power units

    # Create a new model
    model = gp.Model("reduced_unit_commitment")

    # Suppress output if verbose is False
    if not verbose:
        model.setParam('OutputFlag', 0)

    # Add binary decision variables for turning the units on/off
    y = model.addVars(n_units, vtype=GRB.BINARY, name="y")

    # Objective function: minimize total cost
    total_cost = gp.quicksum( reduced_uc.unit_costs[i]*y[i] for i in range(n_units))
    model.setObjective(total_cost, GRB.MINIMIZE)

    # Add constraint: Total power output must meet the load demand L
    model.addConstr(gp.quicksum(reduced_uc.unit_powers[i]*y[i] for i in range(n_units)) >= reduced_uc.load, name="power_balance")

    # Increase precision in Gurobi
    model.setParam('MIPGap', 1e-12)  # Tighter gap for optimality
    model.setParam('FeasibilityTol', 1e-6)  # Tighter feasibility tolerance

    # Optimize the model
    model.optimize()

    if model.status == GRB.OPTIMAL:
        # Extract binary (y) solution
        y_solution = model.getAttr("X", y)  

        y_solution = ''.join('1' if val > 0.5 else '0' for i,val in y_solution.items()) # convert to string

        results = {}
        results['bitstring'] = y_solution
        results['cost'] = model.objVal
        results['runtime'] = model.Runtime
        return results
    
    else:
        raise ValueError("Optimization failed")

# To solve the full unit commitment problem we can use the derivative trick, iterating between solving the reduced UC problem and minimizing over the continuous parameter D.

def derivative_trick_UC_solver(uc, verbose=False):

    # One question remains: How to initialize D? We start with naive bounds upper and lower bounds picked so that D sets all power levels to either their minimum or maximum values.

    D_min = min(uc.B + 2 * uc.C * uc.p_min)
    D_max = max(uc.B + 2 * uc.C * uc.p_max)

    # Then, within this range, we can use a classical optimizer to find the best D.

    def objective(D):
        D = float(D) # Ensure D is a float
        reduced_uc = reduceUC(uc, D) 
        try:
            results = gurobi_reduced_UC_solver(reduced_uc, verbose=verbose)
            return results['cost']
        # If the reduced UC solver fails (happens when D is set small enough there is no valid solution), return a large cost
        except ValueError:
            return 1e12
        
    res = minimize_scalar(objective, bounds=(D_min, D_max), method='bounded')

    # A tweak here -- res only finds local minima, so we add some bins to encourage search from a variety of starting points. This can be improved -- either by better choice of D_min and D_max (note right now values ignore the load completely), or by using a more sophisticated global optimization method. Heuristically, optimal values seem to come from the first few bins. 

    # This loop is what makes the derivative trick solver run slightly slower than the basic Gurobi solver. Again should be fixable if we're time bound. 

    candidates = np.linspace(D_min, D_max, num=10)

    for i in range(len(candidates) - 1):
        cand_res = minimize_scalar(objective, bounds=(candidates[i], candidates[i+1]), method='bounded')
        if cand_res.success and cand_res.fun < res.fun:
            res = cand_res

    if res.success:
        optimal_D = float(res.x)
        reduced_uc = reduceUC(uc, optimal_D)
        final_results = gurobi_reduced_UC_solver(reduced_uc, verbose=verbose)
        final_results['power'] = reduced_uc.unit_powers
        # set power levels to 0 for units that are off
        final_results['power'] = [final_results['power'][i] if final_results['bitstring'][i] == '1' else 0.0 for i in range(len(final_results['power']))]
        return final_results

In [7]:
## Now we compare the solutions of the basic Gurobi solver and the derivative trick solver. Mathematcially they should be the same, but due to numerical issues (and maybe local minima) they differ slightly in practice.

testUC = UnitCommitmentProblem(n_units=100, load_factor=0.2)

basic_solve = gurobi_UC_solver(testUC, verbose=False)
print("Basic Gurobi Solver Results has cost", basic_solve['cost'])

derivative_solve = derivative_trick_UC_solver(testUC, verbose=False)
print("Derivative Trick Solver Results has cost", derivative_solve['cost'])

Basic Gurobi Solver Results has cost 5034.7924983140765
Derivative Trick Solver Results has cost 5036.151584677021


### A Bonus Method for Debugging

In [204]:
def getD(uc, solution):
    """
    Calculate the continuous parameter D based on the current solution.
    
    Args:
        uc (UnitCommitmentProblem): An instance of the UnitCommitmentProblem class.
        solution (dict): The current solution containing 'bitstring' and 'power'.
    Returns:
        D (float): The continuous parameter describing first derivates with respect to unit's power.    
    """
    n_units = len(uc.A)
    
    allmin = True
    allmax = True

    Dvalues = []
    D = 0

    for i in range(n_units):
        if solution['bitstring'][i] == '1':
            if abs(solution['power'][i] - uc.p_min[i]) > 1e-4:
                allmin = False
            if abs(solution['power'][i] != uc.p_max[i]) > 1e-4:
                allmax = False
            if abs(solution['power'][i] - uc.p_min[i]) > 1e-4 and abs(solution['power'][i] != uc.p_max[i]) > 1e-4:
                # print("Solution is neither at min nor max for unit", i)
                D = uc.B[i] + 2 * uc.C[i] * solution['power'][i]
                Dvalues.append(D)                
    
    if allmin:
        print("All units at minimum power")
        D = min(uc.B + 2 * uc.C * uc.p_min)

    if allmax:
        print("All units at maximum power")
        D = max(uc.B + 2 * uc.C * uc.p_max)
                
    # Dvalues should all be identical, but may differ slightly due to numerical precision issues
    return np.median(Dvalues), Dvalues